In [1]:
!pip install numpy pandas scikit-learn nltk keras tensorflow flask


  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade tensorflow
!pip install --upgrade keras



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input




nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luisdavid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!pip install kaggle



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pwd

/Users/luisdavid/Desktop/9A/Mario/U3


In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d kritanjalijain/amazon-reviews

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/__init__.py", line 6, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 258, in <module>
    class KaggleApi(KaggleApi):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 289, in KaggleApi
    os.makedirs(config_dir)
  File "<frozen os>", line 225, in makedirs
OSError: [Errno 30] Read-only file system: '/content'


In [121]:
!unzip amazon-reviews.zip -d amazondata


Archive:  amazon-reviews.zip
replace amazondata/amazon_review_polarity_csv.tgz? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


Este código realiza la preparación y preprocesamiento de datos para entrenar modelos de Machine Learning y Deep Learning a partir de datos de texto (en este caso, reseñas de Amazon)

In [8]:
# Parte 1: Preparación y Preprocesamiento

import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Función para cargar y formatear datos
def load_data(file_path):
    # Cargar datos sin encabezados
    data = pd.read_csv(file_path, header=None)
    # Nombrar columnas basándonos en el formato proporcionado
    data.columns = ["label", "title", "review_text"]
    return data

# Función para limpiar texto
def clean_text(text):
    # Quitar caracteres especiales, puntuaciones y números
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios adicionales
    return text

# Preprocesamiento general
def preprocess_data(data):
    # Limpiar la columna de texto
    data["cleaned_review_text"] = data["review_text"].apply(clean_text)
    # Convertir etiquetas a 0 (negativo) y 1 (positivo)
    data["label"] = data["label"].apply(lambda x: 0 if x == 1 else 1)
    return data

# Vectorización TFIDF
def prepare_tfidf_vectorizer(X_train, X_test):
    vectorizer = TfidfVectorizer(max_features=50000, stop_words='english')
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    return vectorizer, X_train_tfidf, X_test_tfidf

# Preparación para DNN
def prepare_dnn_data(X_train, X_test, max_vocab_size=50000, max_sequence_length=100):
    tokenizer = Tokenizer(num_words=max_vocab_size)
    tokenizer.fit_on_texts(X_train)
    X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_sequence_length)
    X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_sequence_length)
    return tokenizer, X_train_seq, X_test_seq

# Rutas de los datos
train_path = "amazondata/train.csv"
test_path = "amazondata/test.csv"

# Cargar y preprocesar datos
train_data = load_data(train_path)
test_data = load_data(test_path)

train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# División en características y etiquetas
X_train, y_train = train_data["cleaned_review_text"], train_data["label"]
X_test, y_test = test_data["cleaned_review_text"], test_data["label"]

# Preparar datos para TFIDF
vectorizer, X_train_tfidf, X_test_tfidf = prepare_tfidf_vectorizer(X_train, X_test)

# Preparar datos para DNN
tokenizer, X_train_seq, X_test_seq = prepare_dnn_data(X_train, X_test)

print("Datos listos para los modelos:")
print(f"TFIDF - X_train: {X_train_tfidf.shape}, X_test: {X_test_tfidf.shape}")
print(f"DNN - X_train_seq: {X_train_seq.shape}, X_test_seq: {X_test_seq.shape}")




Datos listos para los modelos:
TFIDF - X_train: (3600000, 50000), X_test: (399997, 50000)
DNN - X_train_seq: (3600000, 100), X_test_seq: (399997, 100)


Clasificador TFIDF + Regresión Logística
Incluye:

Preprocesamiento.
Implementación como clase.
Validación cruzada (K-folds).
Generación del informe de métricas.

In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

class LogisticTfidfClassifier:
    def __init__(self, max_features=50000):
        self.vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
        self.model = LogisticRegression(max_iter=1000)

    def train(self, X_train, y_train):
        X_train_tfidf = self.vectorizer.fit_transform(X_train)
        self.model.fit(X_train_tfidf, y_train)

    def predict(self, X_test):
        X_test_tfidf = self.vectorizer.transform(X_test)
        return self.model.predict(X_test_tfidf)

    def cross_validate(self, X, y, cv=5):
        X_tfidf = self.vectorizer.fit_transform(X)
        scores = cross_val_score(self.model, X_tfidf, y, cv=cv, scoring='accuracy')
        return scores.mean()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=["Negative", "Positive"])
        return report

# Entrenamiento y evaluación del clasificador
logistic_tfidf = LogisticTfidfClassifier()
logistic_tfidf.train(X_train, y_train)

# Validación cruzada
cv_score = logistic_tfidf.cross_validate(X_train, y_train)
print(f"Cross-validation accuracy: {cv_score:.4f}")

# Informe de clasificación
logistic_report = logistic_tfidf.evaluate(X_test, y_test)
print("Logistic Regression + TFIDF Classification Report:")
print(logistic_report)


Cross-validation accuracy: 0.8717
Logistic Regression + TFIDF Classification Report:
              precision    recall  f1-score   support

    Negative       0.88      0.87      0.87    199999
    Positive       0.87      0.88      0.87    199998

    accuracy                           0.87    399997
   macro avg       0.87      0.87      0.87    399997
weighted avg       0.87      0.87      0.87    399997



In [131]:
import pickle

# Guarda el vectorizador y el modelo en un archivo
with open("logistic_tfidf_model.pkl", "wb") as file:
    pickle.dump(logistic_tfidf, file)


In [132]:
# Carga el modelo desde el archivo
with open("logistic_tfidf_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)


2. Red Neuronal Profunda (DNN)
Incluye:

Arquitectura como clase.
Entrenamiento y evaluación.
Generación del informe de métricas.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
import numpy as np

class DNNClassifier:
    def __init__(self, vocab_size=50000, sequence_length=100, embedding_dim=128):
        self.model = Sequential([
            Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
            LSTM(128, return_sequences=True),
            Dropout(0.5),
            LSTM(64),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=5, batch_size=32, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return (predictions > 0.5).astype(int).flatten()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=["Negative", "Positive"])
        return report

# Entrenamiento y evaluación del clasificador
dnn_classifier = DNNClassifier(vocab_size=50000, sequence_length=100)
dnn_classifier.train(X_train_seq, y_train, epochs=3, batch_size=64, validation_data=(X_test_seq, y_test))

# Informe de clasificación
dnn_report = dnn_classifier.evaluate(X_test_seq, y_test)
print("DNN Classification Report:")
print(dnn_report)


Epoch 1/3
  212/56250 ━━━━━━━━━━━━━━━━━━━━ 1:48:48 116ms/step - accuracy: 0.6632 - loss: 0.5843

KeyboardInterrupt: 

3. RESEÑA POSITIVA A NEGATIVA..

In [9]:
!pip install transformers datasets pandas torch scikit-learn



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install 'transformers[torch]'



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install "accelerate>=0.26.0"



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import pandas as pd

# Ruta del archivo CSV
file_path = "amazondata/test2.csv"

# Cargar los datos
df = pd.read_csv(file_path)

# Verificar las primeras filas
print(df.head())

# Verificar columnas
print(df.columns)

# Validar que 'input_text' y 'target_text' no tengan valores nulos ni tipos no válidos
df = df.dropna(subset=["original_text", "inverted_text"])  # Eliminar filas con valores nulos
df = df[df["original_text"].apply(lambda x: isinstance(x, str))]  # Asegurarse de que sean cadenas
df = df[df["inverted_text"].apply(lambda x: isinstance(x, str))]  # Asegurarse de que sean cadenas

# Renombrar columnas para tokenización
df = df.rename(columns={"original_text": "input_text", "inverted_text": "target_text"})

# Mostrar datos limpios
print(df.head())

# Guardar los datos limpios 
df.to_csv("amazondata/test2_cleaned.csv", index=False)


   original_polarity                                     original_title  \
0                  2                Great card, better than expected...   
1                  2                                           Great CD   
2                  2  One of the best game music soundtracks - for a...   
3                  1                   Batteries died within a year ...   
4                  2              works fine, but Maha Energy is better   

                                       original_text  inverted_polarity  \
0  Considering the price of this card, its hard t...                1.0   
1  My lovely Pat has one of the GREAT voices of h...                1.0   
2  Despite the fact that I have only played a sma...                1.0   
3  I bought this charger in Jul 2003 and it worke...                2.0   
4  Check out Maha Energy's website. Their Powerex...                1.0   

                                      inverted_title  \
0                  Poor card, worse than e

/var/folders/y4/rsy3c7f57zj3lzswhzhsncf80000gn/T/ipykernel_24144/2740724611.py:7: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [14]:
from datasets import Dataset

# Convertir el DataFrame limpio a Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Dividir en conjunto de entrenamiento y validación
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

# Verificar el formato del Dataset
print(train_dataset[0])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'original_polarity': 2, 'original_title': 'Simple, Durable, Fun game for all ages', 'input_text': "This is an AWESOME game! Almost everyone know tic-tac-toe so it is EASY to learn and quick to play. You can't play just once! The twist is that your pieces are slightly different sizes - just big enough to gobble up your opponent. The first person to make tic-tac-toe wins, but it's not as easy as it looks when you're stuck in the mindset of just making three in a row and forget about the gobbling possibilities! My 4 and 5 year olds will beat me even when I'm trying to win! Excellent beginning critical thinking game. Grandparents loved playing it with the kids too.", 'inverted_polarity': 1.0, 'inverted_title': 'Complex, Fragile, Boring game for all ages', 'target_text': "This is a TERRIBLE game! Most people already know tic-tac-toe, making it DIFFICULT to learn and slow to play. It's frustratingly hard to engage, and the twist with different-sized pieces complicates the gameplay unnecessa

In [15]:
from transformers import T5Tokenizer

# Cargar el tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenización con manejo por lotes
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"], max_length=512, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"], max_length=512, truncation=True, padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenizar los datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Verificar datos tokenizados
print(tokenized_train[0])


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|          | 0/170 [00:00<?, ? examples/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 19/19 [00:00<00:00, 2798.

{'original_polarity': 2, 'original_title': 'Simple, Durable, Fun game for all ages', 'input_text': "This is an AWESOME game! Almost everyone know tic-tac-toe so it is EASY to learn and quick to play. You can't play just once! The twist is that your pieces are slightly different sizes - just big enough to gobble up your opponent. The first person to make tic-tac-toe wins, but it's not as easy as it looks when you're stuck in the mindset of just making three in a row and forget about the gobbling possibilities! My 4 and 5 year olds will beat me even when I'm trying to win! Excellent beginning critical thinking game. Grandparents loved playing it with the kids too.", 'inverted_polarity': 1.0, 'inverted_title': 'Complex, Fragile, Boring game for all ages', 'target_text': "This is a TERRIBLE game! Most people already know tic-tac-toe, making it DIFFICULT to learn and slow to play. It's frustratingly hard to engage, and the twist with different-sized pieces complicates the gameplay unnecessa

In [19]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Cargar el modelo preentrenado
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # Evaluar al final de cada época
    save_strategy="epoch",         # Guardar al final de cada época
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,            # Limitar a 2 checkpoints guardados
    load_best_model_at_end=True,   # Cargar el mejor modelo al final
    logging_dir="./logs",          # Directorio para los logs
)


# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/y4/rsy3c7f57zj3lzswhzhsncf80000gn/T/ipykernel_24144/2412738625.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/220 [00:00<?, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
                                                
 10%|█         | 22/220 [00:43<04:21,  1.32s/it]

{'eval_loss': 10.914222717285156, 'eval_runtime': 1.1736, 'eval_samples_per_second': 16.19, 'eval_steps_per_second': 2.556, 'epoch': 1.0}


                                                
 20%|██        | 44/220 [01:32<04:48,  1.64s/it]

{'eval_loss': 5.393222332000732, 'eval_runtime': 1.444, 'eval_samples_per_second': 13.157, 'eval_steps_per_second': 2.077, 'epoch': 2.0}


                                                
 30%|███       | 66/220 [02:42<06:17,  2.45s/it]

...

                                                 
100%|██████████| 220/220 [11:05<00:00,  2.77s/it]

{'eval_loss': 0.33338698744773865, 'eval_runtime': 1.3179, 'eval_samples_per_second': 14.417, 'eval_steps_per_second': 2.276, 'epoch': 10.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 220/220 [11:06<00:00,  3.03s/it]

{'train_runtime': 666.8962, 'train_samples_per_second': 2.549, 'train_steps_per_second': 0.33, 'train_loss': 3.4997555819424715, 'epoch': 10.0}


TrainOutput(global_step=220, training_loss=3.4997555819424715, metrics={'train_runtime': 666.8962, 'train_samples_per_second': 2.549, 'train_steps_per_second': 0.33, 'total_flos': 230081062502400.0, 'train_loss': 3.4997555819424715, 'epoch': 10.0})

In [20]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate_model(model, tokenizer, dataset):
    true_labels = []  # Etiquetas reales
    pred_labels = []  # Etiquetas predichas

    for example in dataset:
        # Texto original y etiqueta real
        input_text = example["input_text"]
        true_label = example["inverted_polarity"]  # Etiqueta verdadera

        # Generar predicción
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Asignar etiquetas: Si el texto generado contiene algo positivo/negativo, ajustamos la polaridad
        predicted_label = 2 if "great" in predicted_text.lower() or "amazing" in predicted_text.lower() else 1

        true_labels.append(true_label)
        pred_labels.append(predicted_label)

    # Calcular métricas
    accuracy = accuracy_score(true_labels, pred_labels)
    report = classification_report(true_labels, pred_labels, target_names=["Negative", "Positive"])
    return accuracy, report


In [21]:
import torch

# Verificar el dispositivo
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Usando dispositivo: {device}")


Usando dispositivo: mps


In [22]:
# Configurar el dispositivo
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Mover el modelo al dispositivo
model.to(device)

def evaluate_model(model, tokenizer, dataset):
    true_labels = []
    pred_labels = []

    for example in dataset:
        input_text = example["input_text"]
        true_label = example["inverted_polarity"]

        # Generar predicción
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Asignar etiquetas
        predicted_label = 2 if "great" in predicted_text.lower() or "amazing" in predicted_text.lower() else 1

        true_labels.append(true_label)
        pred_labels.append(predicted_label)

    accuracy = accuracy_score(true_labels, pred_labels)
    report = classification_report(true_labels, pred_labels, target_names=["Negative", "Positive"])
    return accuracy, report


In [23]:
# Evaluar modelo
accuracy, report = evaluate_model(model, tokenizer, tokenized_val)
print(f"Cross-validation accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)


Cross-validation accuracy: 0.6316
Classification Report:
              precision    recall  f1-score   support

    Negative       0.63      1.00      0.77        12
    Positive       0.00      0.00      0.00         7

    accuracy                           0.63        19
   macro avg       0.32      0.50      0.39        19
weighted avg       0.40      0.63      0.49        19



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [133]:
import pickle

# Guardar el modelo y el tokenizador
with open("polaridadinvertida1.pkl", "wb") as file:
    pickle.dump({"model": model, "tokenizer": tokenizer}, file)

print("Modelo y tokenizador guardados exitosamente como 'polaridadinvertida1.pkl'")


Modelo y tokenizador guardados exitosamente como 'polaridadinvertida1.pkl'


In [134]:
# Cargar el modelo y el tokenizador desde el archivo
with open("polaridadinvertida1.pkl", "rb") as file:
    data = pickle.load(file)

# Extraer el modelo y el tokenizador
loaded_model = data["model"]
loaded_tokenizer = data["tokenizer"]

print("Modelo y tokenizador cargados exitosamente.")


Modelo y tokenizador cargados exitosamente.


4. ENCORER- DECORER.

In [24]:
!pip install transformers datasets torch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Carga y Preprocesamiento de Datos

Dado que tu archivo CSV es muy pesado, cargaremos solo una parte de él utilizando el parámetro nrows de pandas. Supongamos que deseas trabajar con las primeras 10,000 filas.

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import os


Cargar una Parte del CSV

In [34]:
# Definir la ruta al archivo CSV
csv_path = os.path.join('amazondata', 'test2.csv')

# Cargar solo las primeras 10000 filas
num_samples = 10000
data = pd.read_csv(csv_path, nrows=num_samples)

# Mostrar las primeras filas para verificar
print(data.head())


   original_polarity                                     original_title  \
0                  2                Great card, better than expected...   
1                  2                                           Great CD   
2                  2  One of the best game music soundtracks - for a...   
3                  1                   Batteries died within a year ...   
4                  2              works fine, but Maha Energy is better   

                                       original_text  inverted_polarity  \
0  Considering the price of this card, its hard t...                1.0   
1  My lovely Pat has one of the GREAT voices of h...                1.0   
2  Despite the fact that I have only played a sma...                1.0   
3  I bought this charger in Jul 2003 and it worke...                2.0   
4  Check out Maha Energy's website. Their Powerex...                1.0   

                                      inverted_title  \
0                  Poor card, worse than e

# Seleccionar las columnas de interés
input_texts = data['original_title'].astype(str).tolist()
target_texts = data['inverted_title'].astype(str).tolist()


In [35]:
# Seleccionar las columnas de interés
input_texts = data['original_title'].astype(str).tolist()
target_texts = data['inverted_title'].astype(str).tolist()


4. Agregar Tokens de Inicio y Fin de Secuencia
Es una buena práctica agregar tokens especiales para indicar el inicio y fin de las secuencias.

In [36]:
START_TOKEN = '<start>'
END_TOKEN = '<end>'

target_texts = [START_TOKEN + ' ' + text + ' ' + END_TOKEN for text in target_texts]


5. Crear el Conjunto de Caracteres
python
Copiar código


In [37]:
# Crear un conjunto de caracteres únicos para las entradas y las salidas
input_characters = set(''.join(input_texts))
target_characters = set(''.join(target_texts))

# Ordenar los caracteres para asignar índices consistentes
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# Definir la longitud máxima de las secuencias
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Número de tokens de entrada:', num_encoder_tokens)
print('Número de tokens de salida:', num_decoder_tokens)
print('Longitud máxima de secuencia de entrada:', max_encoder_seq_length)
print('Longitud máxima de secuencia de salida:', max_decoder_seq_length)


Número de tokens de entrada: 101
Número de tokens de salida: 58
Longitud máxima de secuencia de entrada: 125
Longitud máxima de secuencia de salida: 87


6. Crear Mapeos de Caracteres a Índices
python
Copiar código


In [38]:
# Crear diccionarios de mapeo de caracteres a índices
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

# Crear listas de caracteres para recuperar los caracteres a partir de los índices
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}


7. Vectorizar los Datos
Convertiremos las secuencias de caracteres en matrices binarizadas (one-hot encoding).

In [39]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        if t < max_encoder_seq_length:
            encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        if t < max_decoder_seq_length:
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                # decoder_target_data está adelantado por un timestep respecto a decoder_input_data
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.


**Definición del Modelo Codificador-Decodificador**

Ahora, construiremos el modelo utilizando LSTM para el codificador y el decodificador.

In [40]:
# Definir los hiperparámetros
latent_dim = 256  # Dimensionalidad latente del espacio de codificación

# Definir el codificador
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Descartar 'encoder_outputs' y solo mantener los estados
encoder_states = [state_h, state_c]

# Definir el decodificador
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Definir el modelo que se entrenará
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar el modelo
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


Entrenamiento del Modelo

In [42]:
# Definir los parámetros de entrenamiento
batch_size = 64
epochs = 25

# Entrenar el modelo
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)


Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 278ms/step - loss: 0.1353 - val_loss: 0.0840
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 300ms/step - loss: 0.1318 - val_loss: 0.0817
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 298ms/step - loss: 0.1303 - val_loss: 0.0813
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 292ms/step - loss: 0.1293 - val_loss: 0.0804
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 290ms/step - loss: 0.1282 - val_loss: 0.0803
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 293ms/step - loss: 0.1276 - val_loss: 0.0799
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 286ms/step - loss: 0.1267 - val_loss: 0.0797
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 282ms/step - loss: 0.1261 - val_loss: 0.0799
Epoch 9/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 284ms/step - loss: 0.1258 - val_loss: 0.0799
Epoch 10/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 278ms/step - loss: 0.1257 - val_loss: 0.0796
Epoch 11/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 281ms/step - loss: 0.1249 - val_loss: 0.0791
Epoch 12/25
125/125

Definir Tokens de Inicio y Fin como Caracteres Únicos

En lugar de usar '<start>' y '<end>', utilizaremos caracteres únicos como '\t' y '\n'.

In [46]:
# Definir los tokens especiales como caracteres únicos
START_TOKEN = '\t'  # Carácter de tabulación
END_TOKEN = '\n'    # Carácter de salto de línea


Actualizar las Secuencias de Destino

In [47]:
# Agregar los tokens de inicio y fin a las secuencias de destino
target_texts = [START_TOKEN + ' ' + text + ' ' + END_TOKEN for text in target_texts]


Asegurarse de Incluir los Tokens en el Conjunto de Caracteres

Al definir target_characters, ya se incluirán los caracteres de inicio y fin, ya que hemos agregado START_TOKEN y END_TOKEN a target_texts.

In [48]:
# Crear un conjunto de caracteres únicos para las entradas y las salidas
input_characters = set(''.join(input_texts))
target_characters = set(''.join(target_texts))

# Asegurarse de que los tokens especiales estén incluidos
print(f"Caracteres de destino incluidos: {START_TOKEN} y {END_TOKEN}")


Caracteres de destino incluidos: 	 y 



Verificar la Presencia de los Tokens en el Diccionario

In [49]:
# Crear diccionarios de mapeo de caracteres a índices
target_token_index = {char: i for i, char in enumerate(target_characters)}

# Verificar que los tokens especiales estén en el diccionario
assert START_TOKEN in target_token_index, "El token de inicio no está en target_token_index."
assert END_TOKEN in target_token_index, "El token de fin no está en target_token_index."

print("Tokens especiales correctamente incluidos en target_token_index.")


Tokens especiales correctamente incluidos en target_token_index.


Configuración para la Inferencia

Una vez entrenado el modelo, configuraremos modelos separados para el codificador y el decodificador que nos permitirán generar secuencias.

In [43]:
# Modelo del codificador para la inferencia
encoder_model = Model(encoder_inputs, encoder_states)

# Crear tensores de entrada para los estados del decodificador
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Obtener las salidas del decodificador para la inferencia
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo del decodificador para la inferencia
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


Función de Muestreo

Implementaremos una función que siga los pasos que has descrito para generar secuencias.

In [50]:
def decode_sequence(input_seq):
    # 1) Codificar la secuencia de entrada en vectores de estado
    states_value = encoder_model.predict(input_seq)

    # 2) Inicializar la secuencia de destino con el token de inicio
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index[START_TOKEN]] = 1.

    # Inicializar la condición de parada y la oración decodificada
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        # 3) Pasar los vectores de estado y la secuencia de destino al decodificador
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 4) Muestrear el siguiente carácter usando argmax
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 5) Comprobar si hemos alcanzado el token de fin de secuencia o el límite de caracteres
        if (sampled_char == END_TOKEN or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 6) Actualizar la secuencia de destino (un carácter) y los estados
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence



Finalmente, probaremos el modelo con algunas secuencias de entrada.

python
Copiar código


In [51]:
for seq_index in range(10):  # Probar con las primeras 10 muestras
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Entrada:', input_texts[seq_index])
    print('Salida Decodificada:', decoded_sentence)
    print('Salida Real:', target_texts[seq_index])
    print('---')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━

In [52]:
!pip install scikit-learn pandas



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [54]:
# Guardar el modelo completo
model.save('modelo_codificador_decodificador.h5')

# Cargar el modelo
# model = tf.keras.models.load_model('modelo_codificador_decodificador.h5')


In [58]:
print(f"Longitud de y_true: {len(y_true)}")
print(f"Longitud de y_pred: {len(y_pred)}")


Longitud de y_true: 10000
Longitud de y_pred: 4


In [62]:
# Compilar el modelo con la métrica de exactitud
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [65]:
def calculate_sequence_accuracy(encoder_input_data, target_texts, num_samples=1000):
    correct = 0
    total = num_samples
    
    for i in range(num_samples):
        input_seq = encoder_input_data[i: i + 1]
        decoded_sentence = decode_sequence(input_seq)
        
        # Obtener la secuencia de referencia sin tokens especiales
        reference_sentence = target_texts[i].replace(START_TOKEN + ' ', '').replace(' ' + END_TOKEN, '')
        
        if decoded_sentence.strip() == reference_sentence.strip():
            correct += 1
        
        if (i + 1) % 100 == 0:
            print(f'Procesadas {i + 1} / {num_samples} secuencias.')
    
    accuracy = correct / total
    print(f'Exactitud por secuencia: {accuracy * 100:.2f}%')
    return accuracy


In [68]:
# Definir el número de muestras para evaluar
num_samples = 50  # Puedes ajustar este número según tus necesidades

# Calcular la exactitud por secuencia
sequence_accuracy = calculate_sequence_accuracy(encoder_input_data, target_texts, num_samples=num_samples)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━